# Graph exploring

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
from pyld import jsonld
import json
import pandas as pd
import io
import glob
import tempfile
import os
import pyoxigraph
import requests
import advertools as adv
import logging 
from scrapy.utils.log import configure_logging

from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML 
import networkx as nx
from ipysigma import Sigma
from SPARQLWrapper import SPARQLWrapper, JSON
from hashlib import sha256
from io import StringIO

import datasets
import pandas as pd
from datasets import load_dataset
import torch
# from relik import Relik
# from relik.inference.data.objects import RelikOutput
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.INFO)
# logging.getLogger('relik').setLevel(logging.INFO)

In [ ]:
logging.getLogger("urllib3").setLevel(logging.ERROR)
logging.getLogger("requests").setLevel(logging.ERROR)
configure_logging(install_root_handler=False)
logging.getLogger("scrapy").setLevel(logging.ERROR)

In [ ]:
store = pyoxigraph.Store()  #  memory store  
store = pyoxigraph.Store(path="./store")  # disk based store
mime_type = "application/n-quads"   # application/n-triples or application/n-quads if you are loading those

In [ ]:
store.clear()

In [ ]:
sourcesOLD = glob.glob("../datastore/data/**/meta_json-ld*.json") + glob.glob("../datastore/data/**/**/meta_json-ld*.json")
sourcesOLD

In [ ]:

sourcesOLD = [
    "../datastore/data/global_pm25_concentration_1998_2016/meta_json-ld_global_pm25_concentration_1998_2016.json",
    # "../datastore/data/ma_2018_svi_tract/meta_json-ld_ma_2018_svi_tract.json",
    # "../datastore/data/tz_1984_copernicus_avg_temp/meta_json-ld_tz_1984_copernicus_avg_temp.json",
    # "../datastore/data/us_2018_svi_county/meta_json-ld_us_2018_svi_county.json",
    # "../datastore/data/tz_2022_nbs/tz_2022_nbs_districts/meta_json-ld_tz_2022_nbs_districts.json",
    # "../datastore/data/tz_2022_nbs_magu_district/meta_json-ld_tz_2022_nbs_magu_district.json",
    # "../datastore/data/tz_magu_dem_srtm/meta_json-ld_tz_magu_dem_srtm.json",
    # "../datastore/data/tz_populated_places_osm/meta_json-ld_tz_populated_places_osm.json",
    # "../datastore/data/tz_regions_osm/meta_json-ld_tz_regions_osm.json"
]

In [ ]:
# NOT USED
def custom_document_loader(url, options):
    """Custom document loader that returns empty context for invalid URLs"""
    if url == '@context':
        # Return an empty context document
        return {
            'contextUrl': None,
            'documentUrl': url,
            'document': {}
        }
    # For other URLs, you might want to handle them differently
    # or let the default loader handle them
    raise jsonld.JsonLdError(
        f'URL could not be loaded: {url}',
        'jsonld.LoadDocumentError',
        {'url': url}
    )

In [ ]:
sources = glob.glob("../datastore/nquads/*.nq")
sources.remove('../datastore/nquads/meta_json-ld_tz_magu_dem_srtm.nq')
sources

In [ ]:
sources = [
    "./data/meta_json-ld_global_pm25_concentration_1998_2016.nq",
    "./data/meta_json-ld_ma_2018_svi_tract.nq",
    "./data/meta_json-ld_tz_1984_copernicus_avg_temp.nq",
    "./data/meta_json-ld_tz_2022_nbs_districts.nq",
    "./data/meta_json-ld_tz_2022_nbs_magu_district.nq",
    # "./data/meta_json-ld_tz_magu_dem_srtm.nq",
    "./data/meta_json-ld_tz_populated_places_osm.nq",
    "./data/meta_json-ld_tz_regions_osm.nq",
    "./data/meta_json-ld_us_2018_svi_county.nq"
]



In [ ]:
# Read a JSON file and parse it
for s in sources:
    with open(s) as nq_file:
        print(s)
        store.load(nq_file, pyoxigraph.RdfFormat.N_QUADS, base_iri=None, to_graph=None)

In [ ]:
#Read a JSON file and parse it
for s in sourcesOLD:
    print(s)
    doc_array = None
    with open(s) as json_file:
      doc_array = json.load(json_file)

    for ind, x in enumerate(doc_array):
        doc = json.loads(json.dumps(doc_array[x]))  # x is a dict so we need to typecast
        normalized = jsonld.normalize(doc, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})
        ns = io.StringIO(normalized)
        store.load(ns, mime_type, base_iri=None, to_graph=None)

In [ ]:
for ind, x in enumerate(doc_array): print(json.dumps(doc_array[x]))

In [ ]:
## NOTE   schema:contentURL changed!!!

qtest = """	PREFIX schema: <https://schema.org/>
SELECT ?s ?o 
WHERE {
      ?s schema:description ?o .
 }
"""

q1 = list(store.query(qtest))

In [ ]:
df1 = pd.DataFrame(q1, columns=["subject",  "object" ])

In [ ]:
df1

In [ ]:
term_counts = df1['object'].value_counts().reset_index()

# Rename the columns for clarity
term_counts.columns = ['object', 'count']

In [ ]:
term_counts.head(30)

## Visualization section

In [ ]:
#  a == rdf:type  == <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>

qtypetype = """	PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?connectedBy ?target ?sType ?tType
WHERE {
    ?source a ?sType .
    ?target a ?tType .
    ?source ?connectedBy ?target .
}
"""

# qtypetype = """	PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
# PREFIX schema: <https://schema.org/>
#
# SELECT DISTINCT ?source ?sType ?kw
# WHERE {
#     ?source a ?sType .
#     ?source schema:keywords ?kw .
# }
# """

q2 = list(store.query(qtypetype))

In [ ]:
# df2 = pd.DataFrame(q2, columns=["source", "sType", "keyword"])
df2 = pd.DataFrame(q2, columns=["source", "connecteBy", "target", "sType", "tType"])


In [ ]:
df2 = df2.astype(str) # convert oxigraph types to string

In [ ]:
gml = GraphML()
gml.directed = False

for index, row in df2.iterrows():
    n1 = gml.add_node(row['source'])
    n1['type'] = row['sType']
    
    n2 = gml.add_node(row['target'])
    n2['type'] = "tType"
 
    e1 = gml.add_edge(n1, n2)

# Create a temporary file
with tempfile.NamedTemporaryFile(delete=False, mode='w') as temp_file:
    temp_filename = temp_file.name
    # Write the GraphML content to the temporary file
    parser = GraphMLParser()
    parser.write(gml, temp_filename)
    
g = nx.read_graphml(temp_filename)

os.remove(temp_filename)


In [ ]:
# optional write the GML out
# fname = "./rdfGraphs2.graphml"
# parser = GraphMLParser()
# parser.write(gml, fname)


In [ ]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=0
)


## test relik against the descriptions

In [ ]:
def clean_memory():
    # clean up a bit
    import gc

    relik = None
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
def my_function(row):
  # Perform calculations or transformations on the row
  relik_out: RelikOutput = relik(row['description'])
  return relik_out

In [ ]:
tqdm.pandas()

In [ ]:
clean_memory()

In [ ]:
relik = Relik.from_pretrained(
    "sapienzanlp/relik-entity-linking-small",
    # device="cuda" if torch.cuda.is_available() else "cpu",
    # precision="fp16" if torch.cuda.is_available() else "fp32",
    device="cpu", 
    precision="fp32",  
    skip_metadata=True,  # don't load index metadata to keep low memory requirements
    verbose=False  # Add this line to suppress debug output
)

# 
# relik = Relik.from_pretrained(
#     "relik-ie/relik-relation-extraction-small",
#     device="cuda" if torch.cuda.is_available() else "cpu",
#     precision="fp16" if torch.cuda.is_available() else "fp32",
#     skip_metadata=True,  # don't load index metadata to keep low memory requirements
# )


In [ ]:

qdesc = """	PREFIX schema: <https://schema.org/>
SELECT ?s ?o 
WHERE {
      ?s schema:description ?o .
 }
"""

q3 = list(store.query(qdesc))

In [ ]:
df3 = pd.DataFrame(q3, columns=["source", "description"])

In [ ]:
df3 = df3.astype(str) # convert oxigraph types to string

In [ ]:
df3

In [ ]:
df_b = pd.DataFrame()

In [ ]:
for index, row in tqdm(df3.iterrows(), total=df3.shape[0]):
  result_dict = my_function(row)
  df = pd.DataFrame(result_dict.spans)
  # df = pd.DataFrame(result_dict.triplets)
  df['iri'] = row['source']
  df_b = pd.concat([df_b, df], ignore_index=True)

In [ ]:
# takes 3.5 hrs to build
df_b.to_parquet('df_b.parquet')

In [ ]:
count_series = df_b['label'].value_counts()



In [ ]:
count_series.to_csv('value_counts.csv', header=True, index_label='Value')


In [ ]:
print(count_series)


In [ ]:
df_b